In [5]:
import requests
try:
    response = requests.post(
        "http://localhost:8000/chat",
        json={"text": "Say hi in one word"}
    )
    response.raise_for_status()
    print(response.json()["response"])
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

Hi
